### Example Exploratory Notebook

Use this notebook to explore the data generated by the pipeline in your preferred programming language.

**Note**: This notebook is not executed as part of the pipeline.

In [0]:
# Create a streaming DataFrame by reading from the profiles table
df = (
  spark.table("profiles_bronze")
    .select("topic", "partition", "offset", "value_str", "timestampType", "timestamp", "ingestTime")
    .withColumn(
      "latency_ms"
      ,unix_millis(col("ingestTime")) - unix_millis(col("timestamp"))
    )
    .orderBy(col("timestamp"), ascending=False)
    .limit(1000)
  )

# Display the streaming DataFrame
display(df)

In [0]:
# Calculate the average latency_ms from streaming_df
average_latency_ms_df = df.groupBy().avg("latency_ms")

# Display the result
display(average_latency_ms_df)

In [0]:
df = (
  spark.read.option("readChangeFeed", "true")
  .option("startingVersion", 11)
  .table(f"profiles_bronze")
  .filter("_change_type = 'delete'")
)
display(df)

In [0]:
%sql
DESCRIBE EXTENDED profiles AS JSON;

In [0]:
%sql 
FROM profiles_scd2 |>
AGGREGATE COUNT(*) as rcrd_cnt 
GROUP BY __START_AT._commit_version 

In [0]:
%fs ls s3://adc-lv-dev-devops.libreview-data-migration/patient-1_Measurement/patient-1_Measurement/

In [0]:
%fs  head -100 s3://adc-lv-dev-devops.libreview-data-migration/patient-1_Measurement/patient-1_Measurement/patient-1_Measurement_1.tsv

In [0]:
%fs ls s3://adc-lv-dev-devops.libreview-data-migration/